# KoBERT를 이용해 학습적용

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import sentencepiece as spm

c:\Users\ysj_1\miniconda3\envs\ds_study\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
c:\Users\ysj_1\miniconda3\envs\ds_study\lib\site-packages\transformers\generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
c:\Users\ysj_1\miniconda3\envs\ds_study\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [20]:
df_star.to_csv('df_star.csv',index=False)
df_no_star.to_csv('df_no_star.csv',index=False)

In [21]:
df_star = pd.read_csv('df_star.csv')
df_no_star = pd.read_csv('df_no_star.csv')

In [24]:
df_star = df_star[['review_content',	'rating']]

In [25]:
df_no_star = df_no_star[['review_content',	'rating']]

In [27]:
df_star.reset_index(drop=True, inplace=True)
df_no_star.reset_index(drop=True, inplace=True)

In [28]:
from sklearn.model_selection import train_test_split

# df_star 데이터프레임을 train과 test로 나누기 위해 train_test_split 함수를 사용한다.
# test_size는 test 데이터셋의 비율을 의미한다.
# random_state는 난수 발생 시드값으로, 같은 시드값을 사용하면 같은 결과를 얻을 수 있다.
train, test = train_test_split(df_star, test_size=0.2, random_state=42,stratify=df_star['rating'])



In [29]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)


In [30]:
len(train),len(test)

(197857, 49465)

In [31]:
test['rating'].value_counts(),train['rating'].value_counts()

(1    43125
 0     6340
 Name: rating, dtype: int64,
 1    172496
 0     25361
 Name: rating, dtype: int64)

In [39]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 


def clean(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()


import re


def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s\n]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    text = re.sub('\n', '.', string=text)
    return text 

In [51]:
df_star = df_star.copy()

In [52]:
df_star.loc[:,'review_content'] = df_star['review_content'].apply(lambda x : clean_str(x)).copy()
df_star

,review_content,rating
0,최고입니다 수년째 단골인데 리뷰는 처음 써보네요 이곳저곳에서 장어 많이 먹어봤는데 ...,1
1,맛은 좋은데 일하시는 여자 분 진짜 매너 없음 잠바 입고 써빙해줌 목도리 자락...,0
2,양평 유명 장어맛집이라고 가봤던곳은 흙냄새 같은게 나서 많이 먹을수가 없었는데 여기...,1
3,맛있어요,1
4,상봉역 맛집 쿠시꼬시 소금구이 추천합니다,1
...,...,...
247317,주인할머니께서 맛있게 해주세요,1
247318,비빔국수가 빨간양념 맛보다 참기름 맛이 좀 더 강한 비빔국수예요 그래서 같이 주신...,1
247319,국수 맛나서 종종 방문합니다,1
247320,멸치국수 삼성역 구수하고 정겨운 맛이에요 곱빼기도 파는데 담에는 곱빼기 먹을래요,1


In [54]:
df_no_star

,review_content,rating
0,밑반찬도 맛있고 특히 장어가 맛있어요,별점없음
1,맛있어요 반찬도 맛있구요,별점없음
2,좋아요,별점없음
3,좋아요,별점없음
4,맛있어용,별점없음
...,...,...
1011987,뇸뇸,별점없음
1011988,맛납니다.,별점없음
1011989,NaN,별점없음
1011990,맛있어요,별점없음


In [56]:
df_no_star = df_no_star.drop_duplicates(subset='review_content')
df_no_star.dropna(inplace=True)
df_no_star.loc[:,'review_content'] = df_no_star['review_content'].apply(lambda x : clean_str(x)).copy()
df_no_star

,review_content,rating
0,밑반찬도 맛있고 특히 장어가 맛있어요,별점없음
1,맛있어요 반찬도 맛있구요,별점없음
2,좋아요,별점없음
3,좋아요,별점없음
4,맛있어용,별점없음
...,...,...
1011973,먹어본것중최고입돠.또먹구시퍼요,별점없음
1011974,신도림에서 곱창 먹고 싶으면 이리로 옵니다.신선하고 맛있어요,별점없음
1011977,최악의 경험 사람 없는 시간대에 갔음에도 최악이었던 서비스와 심지어 맛까지 없었던 ...,별점없음
1011979,6년째 다니고 있는 내 최애곱창님 언제나 맛있고 그 맛에 감동과 함께 과음하게 됩니다,별점없음


## 버트 인풋 만들기

한글 데이터를 분석하려면, 100개가 넘는 언어에 대해 훈련된 버트를 사용해야 합니다.
이번에는 한국어 데이터로 훈련되었고, SKT에서 만든 KoBERT를 사용하도록 하겠습니다.
모델을 로드하기에 앞서, 토크나이저를 불러오도록 하겠습니다.
huggingface에서는 아주 쉽게 토크나이저를 불러올 수 있습니다.
https://github.com/monologg/KoBERT-NER 에서 kobert를 tokenize 할 수 있는 코드를 가져왔습니다.

In [32]:
# KoBERT 토크나이저를 활용 

import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece `_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [33]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

loading file tokenizer_78b3253a26.model from cache at C:\Users\ysj_1/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\tokenizer_78b3253a26.model
loading file vocab.txt from cache at C:\Users\ysj_1/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\ysj_1/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\ysj_1/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\config.json
Model config BertConfig {
  "_name_or_path": "monologg/kobert",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,

In [34]:
def convert_data(data_df):
    global tokenizer

    SEQ_LEN = 64  # SEQ_LEN : 버트에 들어갈 인풋의 길이

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(
            data_df[DATA_COLUMN][i], truncation=True, padding='max_length', max_length=SEQ_LEN)

        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의


def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y


SEQ_LEN = 64
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "review_content"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "rating"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)


100%|██████████| 197857/197857 [00:29<00:00, 6797.50it/s]


In [35]:
train_x

[array([[   2, 3194, 5561, ...,    1,    1,    1],
        [   2, 4996, 1967, ...,    1,    1,    1],
        [   2,  517, 5960, ..., 6903, 3255,    3],
        ...,
        [   2,  517,    9, ...,    1,    1,    1],
        [   2, 2482, 5859, ...,    1,    1,    1],
        [   2,  517, 6831, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [151]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

100%|██████████| 20540/20540 [00:03<00:00, 6194.09it/s]


## 버트를 활용한 감성분석 모델 만들기

In [23]:
model = TFBertModel.from_pretrained(
    "monologg/kobert", from_pt=True)  # monologg/kobert 모델 불러오기


# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input(
    (SEQ_LEN,), dtype=tf.int32, name='input_word_ids')  # 입력 토큰 정의
mask_inputs = tf.keras.layers.Input(
    (SEQ_LEN,), dtype=tf.int32, name='input_masks')  # 입력 마스크 정의
segment_inputs = tf.keras.layers.Input(
    (SEQ_LEN,), dtype=tf.int32, name='input_segment')  # 입력 세그먼트 정의


# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model(
    [token_inputs, mask_inputs, segment_inputs])  # bert 모델의 출력 정의


loading configuration file config.json from cache at C:\Users\ysj_1/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}

loading weights file pytorch_model.bin from cache at C:\Users\ysj_1/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\pytorch_model.bin
Loading PyTorch weights from C:\Users\ysj_1\.cache\huggin

In [24]:
bert_outputs

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 64, 768) dtype=float32 (created by layer 'tf_bert_model')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [25]:
bert_outputs = bert_outputs[1]

In [26]:
# Rectified Adam 옵티마이저 사용
# tensorflow_addons 패키지 설치
import tensorflow_addons as tfa

# 총 batch size * 4 epoch = 2344 * 4
# Rectified Adam 옵티마이저 생성
opt = tfa.optimizers.RectifiedAdam(
    lr=5.0e-5,  # learning rate 설정
    total_steps=2344*2,  # 총 스텝 수 설정
    warmup_proportion=0.1,  # warmup 비율 설정
    # warmup 스텝 수 설정
    #warmup_steps=int((2344*2) * 0.1)
)


c:\Users\ysj_1\miniconda3\envs\ds_study\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\ysj_1\miniconda3\envs\ds_study\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue

In [27]:

sentiment_drop = tf.keras.layers.Dropout(
    0.5)(bert_outputs)  # BERT 출력값에 드롭아웃 적용
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(
    stddev=0.02))(sentiment_drop)  # 드롭아웃 적용된 BERT 출력값에 fully connected layer 적용
sentiment_model = tf.keras.Model(
    [token_inputs, mask_inputs, segment_inputs], sentiment_first)  # 입력값과 출력값을 지정하여 모델 생성
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[
                        'acc'])  # 모델 컴파일, optimizer는 Adam, loss는 binary crossentropy, 평가 지표는 accuracy


In [157]:
# GPU를 사용하기 위해 런타임 유형을 GPU로 변경

# 필요한 라이브러리 import
import tensorflow as tf

# GPU를 사용하기 위한 설정
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# 모델 학습
with tf.device('/device:GPU:0'):
  sentiment_model.fit(train_x, train_y, epochs=2, shuffle=True,
                      batch_size=64, validation_data=(test_x, test_y))


Epoch 1/2
1284/1284 [==============================] - 21795s 17s/step - loss: 0.3179 - acc: 0.8812 - val_loss: 0.2222 - val_acc: 0.9168
Epoch 2/2
1284/1284 [==============================] - 22433s 17s/step - loss: 0.2081 - acc: 0.9232 - val_loss: 0.2108 - val_acc: 0.9194


In [158]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []

    for i in tqdm(range(len(data_df))):  # 데이터프레임의 길이만큼 반복문 실행

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN,
                                 truncation=True, padding='max_length')  # 문장을 토큰화하여 인코딩
        num_zeros = token.count(0)  # 0의 개수를 세어 num_zeros에 저장
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros  # mask 생성
        segment = [0]*SEQ_LEN  # segment 생성

        tokens.append(token)  # tokens 리스트에 token 추가
        segments.append(segment)  # segments 리스트에 segment 추가
        masks.append(mask)  # masks 리스트에 mask 추가

    tokens = np.array(tokens)  # tokens 리스트를 numpy 배열로 변환
    masks = np.array(masks)  # masks 리스트를 numpy 배열로 변환
    segments = np.array(segments)  # segments 리스트를 numpy 배열로 변환
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의


def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(
        str)  # 데이터프레임의 DATA_COLUMN 열을 문자열로 변환
    data_x = predict_convert_data(data_df)  # convert_data 함수를 이용하여 데이터 전처리
    return data_x


In [159]:

test_set = predict_load_data(test)

100%|██████████| 20540/20540 [00:04<00:00, 4751.73it/s]


In [160]:
test_set

[array([[   2, 1967, 7086, ...,    1,    1,    1],
        [   2,  517, 6040, ...,    1,    1,    1],
        [   2,  532, 5789, ...,    1,    1,    1],
        ...,
        [   2, 1967, 7141, ...,    1,    1,    1],
        [   2, 1458, 1967, ...,    1,    1,    1],
        [   2, 4656,   54, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [161]:
preds = sentiment_model.predict(test_set)

In [162]:

# 부정이면 0, 긍정이면 1 출력
preds

array([[0.46574444],
       [0.9099399 ],
       [0.3828514 ],
       ...,
       [0.889914  ],
       [0.9962193 ],
       [0.99392766]], dtype=float32)

In [163]:

from sklearn.metrics import classification_report
y_true = test['rating']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.79      0.54      0.64      2746
           1       0.93      0.98      0.95     17794

    accuracy                           0.92     20540
   macro avg       0.86      0.76      0.80     20540
weighted avg       0.91      0.92      0.91     20540



- 긍정에 대해서는 스코어가 높게 나오나 부정에 대해서는 스코어가 낮게 나온다.
- 데이터의 불균형의 문제가 있는 것으로 보인다.

In [165]:

def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN,
                             truncation=True, padding='max_length')

    num_zeros = token.count(0)  # 패딩으로 추가된 0의 개수를 세어준다.
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros  # 패딩 부분은 0으로 마스킹한다.
    segment = [0]*SEQ_LEN  # 문장의 길이만큼 segment를 0으로 채운다.

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]


def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value, 0).item()

    if predict_answer == 0:
      # 예측값이 0일 경우, 부정적인 평가로 판단한다.
      print("(부정 확률 : %.2f) 부정적인 평가입니다." % (1-predict_value))
    elif predict_answer == 1:
      # 예측값이 1일 경우, 긍정적인 평가로 판단한다.
      print("(긍정 확률 : %.2f) 긍정적인 평가입니다." % predict_value)


In [195]:
sentiment_model.save_weights("ckpt")

In [43]:
df_no_star[df_no_star['review_content'].notnull()].iloc[0,0]

'밑반찬도 맛있고 특히 장어가 맛있어요'